# C964 Capstone: House Price Prediction

In this Jupyter notebook, we develop a machine learning model for Crestview Realty to predict house prices. The process begins with data exploration and visualizations to understand the dataset, followed by the construction and valuation of various models. Our aim is to identify the most effective model that provides accurate price predictions, for usage within our application.


In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt
from IPython.display import HTML, display
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error, mean_squared_error, make_scorer
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder, StandardScaler, OneHotEncoder

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

# Read the data
X = pd.read_csv('./data/train.csv', index_col='Id')

## Data Exploration

In the upcoming section, we begin with data exploration. This initial phase is needed for gaining an understanding of  the dataset's characteristics. We start with basic statistical analysis and examining distributions to gain an initial overview. Additionally, we separate the numerical and categorical variables to analyze these datatypes individually. This is followed by a missing value analysis, important for assessing the data quality. Finally, we focus on outlier detection, utilizing individual scatter plots for each numerical feature to identify anomalies. This approach lays the groundwork for informed data processing and model development in later sections.

### Basic Statistical & Distribution Analysis

In [ ]:
X.head() # Overview of the data

In [ ]:
X.describe() # Descriptive statistics for each column

In [ ]:
sns.histplot(X, x=X['SalePrice'])

We can see in the graph above that SalePrice is positively skewed. We will likely want to log transform the data since many algorithms assume that the features follow a normal distribution. 

In [ ]:
# Data types of each column
dtypes_html = X.dtypes.to_frame().rename(columns={0: 'Data Type'}).to_html()
display(HTML(f"""
<div style="max-height: 200px; overflow-y: scroll;">
    {dtypes_html}
</div>
"""))

In [ ]:
numerical_df = X.select_dtypes(include=[np.number])
categorical_df = X.select_dtypes(include=['object'])

In [ ]:
# Number of unique values for each categorical column
categorical_df.nunique().to_frame().rename(columns={0: 'Unique values'})

In [ ]:
# Statistical summary of categorical columns
categorical_df.describe()


In [ ]:
# Histograms for each numerical column
numerical_df.hist(bins=50, figsize=(20,15))
plt.subplots_adjust(hspace=0.5)
plt.show()

### Missing Values Analysis

In [ ]:
missing_values = X.isnull().sum().sort_values(ascending=False).where(lambda x: x > 0).dropna()
missing_values_html = missing_values.to_frame().rename(columns={0: 'Missing Values'}).to_html()
display(HTML(f"""
<div style="max-height: 200px; overflow-y: scroll;">
    {missing_values_html}
</div>
"""))


Some of these features have a large number of missing values. We will likely drop some of these columns in the data preparation phase.

### Outlier Detection

In [ ]:
# Skewness of the numerical columns
skew_html = numerical_df.skew().to_frame().to_html()
display(HTML(f"""
<div style="max-height: 200px; overflow-y: scroll;">
    {skew_html}
</div>
"""))

Skewness is a measure of the asymmetry of the probability distribution of a real-valued random variable about its mean. It can be useful for understanding the direction and relative magnitude of a dataset's deviation from a normal distribution. In this case, a high number could suggest the presence of outliers, and shows us where we might want to investigate. `MiscVal`, `PoolArea`, `3SsnPorch`, `LotArea`, and `LowQualFinSF` all stand out as having higher values, so we will keep this in mind.

#### Scatter Plots

In the following section, we will plot each of the numerical columns in a scatter plot with the `SalePrice` for visual inspection for outliers. If an outlier is identified, we will run a query to get the ID of that item, and make a note of it. If further investigation is needed, we may also calculate the z-scores for the feature.

In [ ]:
plt.scatter(x='MSSubClass', y='SalePrice', data=X)

In [ ]:
plt.scatter(x='LotFrontage', y='SalePrice', data=X)
X.query('LotFrontage > 300') # Outliers: 935, 1299

In [ ]:
plt.scatter(x='LotArea', y='SalePrice', data=X)
X.query('LotArea > 100000')
# Outliers: 250, 314, 336, 707

In [ ]:
stats.zscore(X['LotArea']).sort_values().tail(10)

In [ ]:
plt.scatter(x='OverallQual', y='SalePrice', data=X)
X.query('OverallQual == 10 and SalePrice < 200000') # Outliers: 524, 1299

In [ ]:
plt.scatter(x='OverallCond', y='SalePrice', data=X)
X.query('OverallCond == 2 and SalePrice > 375000') # Outlier: 379 

In [ ]:
plt.scatter(x='YearBuilt', y='SalePrice', data=X)
X.query('YearBuilt < 1900 and SalePrice > 400000') # Outlier: 186

In [ ]:
plt.scatter(x='YearRemodAdd', y='SalePrice', data=X)
X.query('YearRemodAdd < 1970 and SalePrice > 300000') # Potential Outlier: 314

In [ ]:
stats.zscore(X['YearRemodAdd']).sort_values().tail(10)

In [ ]:
plt.scatter(x='MasVnrArea', y='SalePrice', data=X)
X.query('MasVnrArea > 1500') # Outlier: 298

In [ ]:
plt.scatter(x='BsmtFinSF1', y='SalePrice', data=X)
X.query('BsmtFinSF1 > 5000') # Outliers: 1299

In [ ]:
plt.scatter(x='BsmtFinSF2', y='SalePrice', data=X)
X.query('BsmtFinSF2 > 1400') # Outlier: 323

In [ ]:
X.query('BsmtFinSF2 > 400 and SalePrice > 500000') # Potential Outlier: 441

In [ ]:
stats.zscore(X['BsmtFinSF2']).sort_values().tail(15) # there are a lot of higher zscores here

In [ ]:
plt.scatter(x='TotalBsmtSF', y='SalePrice', data=X)
X.query('TotalBsmtSF > 6000') # Outlier: 1299

In [ ]:
plt.scatter(x='BsmtUnfSF', y='SalePrice', data=X)
stats.zscore(X['BsmtUnfSF']).sort_values().tail(15)

In [ ]:
plt.scatter(x='1stFlrSF', y='SalePrice', data=X)

In [ ]:
plt.scatter(x='2ndFlrSF', y='SalePrice', data=X)

In [ ]:
plt.scatter(x='LowQualFinSF', y='SalePrice', data=X)
X.query('LowQualFinSF > 550') # Outlier: 186

In [ ]:
plt.scatter(x='GrLivArea', y='SalePrice', data=X)
X.query('GrLivArea > 4500 and SalePrice < 250000') # Outliers: 524, 1299

In [ ]:
plt.scatter(x='BsmtFullBath', y='SalePrice', data=X)
X.query('BsmtFullBath == 3') # Outlier: 739

In [ ]:
stats.zscore(X['BsmtFullBath']).unique()

In [ ]:
plt.scatter(x='BsmtHalfBath', y='SalePrice', data=X)
X.query('BsmtHalfBath == 2') 

In [ ]:
stats.zscore(X['BsmtHalfBath']).unique() # Outliers: 598, 955 (high zscores)


In [ ]:
plt.scatter(x='FullBath', y='SalePrice', data=X)

In [ ]:
plt.scatter(x='HalfBath', y='SalePrice', data=X)

In [ ]:
plt.scatter(x='BedroomAbvGr', y='SalePrice', data=X)
X.query('BedroomAbvGr == 8') # Outlier: 636

In [ ]:
plt.scatter(x='KitchenAbvGr', y='SalePrice', data=X)
X.query('KitchenAbvGr == 3') # Outliers: 49, 810

In [ ]:
plt.scatter(x='TotRmsAbvGrd', y='SalePrice', data=X)
X.query('TotRmsAbvGrd == 14') # Outlier: 636

In [ ]:
plt.scatter(x='Fireplaces', y='SalePrice', data=X)
stats.zscore(X['Fireplaces']).unique()

In [ ]:
plt.scatter(x='GarageYrBlt', y='SalePrice', data=X)

In [ ]:
plt.scatter(x='GarageCars', y='SalePrice', data=X)

In [ ]:
stats.zscore(X['GarageCars']).unique()

In [ ]:
plt.scatter(x='GarageArea', y='SalePrice', data=X)
stats.zscore(X['GarageArea']).sort_values().tail(10)
# Outliers: 1299, 582, 1191

In [ ]:
plt.scatter(x='WoodDeckSF', y='SalePrice', data=X)
stats.zscore(X['WoodDeckSF']).sort_values().tail(10)
# Potential Outliers: 54, 1460, 1069

In [ ]:
plt.scatter(x='OpenPorchSF', y='SalePrice', data=X)
X.query('OpenPorchSF > 325') 
# Outliers: 1329, 496, 524, 
# Potential Outliers: 584, 855

In [ ]:
stats.zscore(X['OpenPorchSF']).sort_values().tail(10)

In [ ]:
plt.scatter(x='EnclosedPorch', y='SalePrice', data=X)
X.query('EnclosedPorch > 350') # Outliers: 198
# Potential outlier: 748, 1198

In [ ]:
stats.zscore(X['EnclosedPorch']).sort_values().tail(10)

In [ ]:
plt.scatter(x='3SsnPorch', y='SalePrice', data=X)
stats.zscore(X['3SsnPorch']).sort_values().tail(10) 
# high z-scores here - may use feature engineering to combine with other porch features


In [ ]:
plt.scatter(x='ScreenPorch', y='SalePrice', data=X)
stats.zscore(X['ScreenPorch']).sort_values().tail(10) # Potential Outliers: 1329, 1387, 186 

In [ ]:
plt.scatter(x='PoolArea', y='SalePrice', data=X)
stats.zscore(X['PoolArea']).sort_values().tail(10) # Outliers: 1424, 811, 1171, 1183, 1387, 198, 1299

##### Scatter Plot Outlier Detection Results

I have compiled a list of outliers and potential outliers for each feature we have plotted and analyzed. This list will be used in the next stage, when we begin data preparation, to remove homes with outliers from our dataset. This identification is subjective, so I've listed additional details as a reminder for later. It's also important to point out that some of these features could be combined using feature engineering into a single feature, and we would want to view the new features and verify that the outliers still exist. The results are as follows:

* LotFrontage - 935, 1299
* LotArea - 250, 314, 336, 707
* OverallQual - 524, 1299
* OverallCond - 379
* YearBuilt - 186
* YearRemodAdd - maybe 314 (leaning no)
* MasVnrArea - 298
* BsmtFinSF1 - 1299
* BsmtFinSF2 - 323, maybe 441 (leaning no)
* TotalBsmtSF - 1299
* LowQualFinSF - 186
* GrLivArea - 524, 1299
* BsmtFullBath - 739
* BsmtHalfBath - 598, 955 (with only 2, but high z-scores)
- BedroomAbvGr - 636
- KitchenAbvGr - 49, 810
- TotRmsAbvGr - 636
- GarageArea - 1299, 582, 1191 (but not that high z-score)
- WoodDeckSF - maybe 54, 1460, 1069
- OpenPorchSF - 1329, 496, 524, maybe 584, 855
- EnclosedPorch -  198, maybe 748, 1198
- 3SsnPorch - a lot of high z-scores here, unclear which would be outliers
- ScreenPorch - 1329, 1387, 186 
- PoolArea - 1424, 811, 1171, 1183, 1387, 198, 1299

## Data Preparation

In this section, we refine our dataset ensuring it is in a clean format ready for modeling. Our first step is to fill in missing values, using strategies to impute or replace them, and subsequently verifying these changes with categorical plots. Features with a high proportion of missing values are considered for removal. Building on our outlier identification from last section, we will now determine which houses to drop from our data based on being outliers. We then perform feature engineering to construct new variables that could enhance our model. Then, we'll check for correlation between features and remove any features that are too closely related, as they can skew the results. Finally, we separate our categorical data columns to be encoded - either ordinal or one-hot encoding.

### Missing Value Analysis

In the previous step, we looked at the number of missing values per feature. Below, we go through each feature that had missing data, use the `unqiue` function to view the current values, and then fill in the missing value -- often with "None". Then, we plot the data on a categorical box plot to verify the imputation strategy. A helper function is created to fill the values in the dataset.

In [ ]:
def replace_empty_value(col, value):
    """Replaces empty in dataset in the provided column 
    name with the provided value
    
    col = the column name
    value = the replacement value
    """
    X[col].fillna(value, inplace=True)

In [ ]:
replace_empty_value('PoolQC', 'None')
sns.catplot(data=X, x='PoolQC', y='SalePrice', kind='box')

In [ ]:
X['Alley'].unique()

In [ ]:
replace_empty_value('Alley', 'None')

In [ ]:
sns.catplot(data=X, x='Alley', y='SalePrice', kind='box')

In [ ]:
X['Fence'].unique()
sns.catplot(data=X, x='Fence', y='SalePrice', kind='box')
replace_empty_value('Fence', 'None')

In [ ]:
replace_empty_value('MasVnrType', 'None')
replace_empty_value('MasVnrArea', 0)
sns.catplot(data=X, x='MasVnrType', y='SalePrice', kind='box')

In [ ]:
X['FireplaceQu'].unique()
replace_empty_value('FireplaceQu', 'None')

In [ ]:
sns.catplot(data=X, x='FireplaceQu', y='SalePrice', kind='box')

In [ ]:
replace_empty_value('LotFrontage', 0)

In [ ]:
X['GarageCond'].unique()
replace_empty_value('GarageCond', 'None')
sns.catplot(data=X, x='GarageCond', y='SalePrice', kind='box')

In [ ]:
X['GarageType'].unique()
replace_empty_value('GarageType', 'None')
sns.catplot(data=X, x='GarageType', y='SalePrice', kind='box')

In [ ]:
X['GarageQual'].unique()
replace_empty_value('GarageQual', 'None')
sns.catplot(data=X, x='GarageQual', y='SalePrice', kind='box')

In [ ]:
X['GarageFinish'].unique()
replace_empty_value('GarageFinish', 'None')
sns.catplot(data=X, x='GarageFinish', y='SalePrice', kind='box')

In [ ]:
X['BsmtExposure'].unique()
replace_empty_value('BsmtExposure', 'None')
sns.catplot(data=X, x='BsmtExposure', y='SalePrice', kind='box')

In [ ]:
replace_empty_value('BsmtFinType2', 'None')
sns.catplot(data=X, x='BsmtFinType2', y='SalePrice', kind='box')

In [ ]:
replace_empty_value('BsmtFinType1', 'None')
sns.catplot(data=X, x='BsmtFinType1', y='SalePrice', kind='box')

In [ ]:
replace_empty_value('BsmtCond', 'None')
sns.catplot(data=X, x='BsmtCond', y='SalePrice', kind='box')

In [ ]:
replace_empty_value('BsmtQual', 'None')
sns.catplot(data=X, x='BsmtQual', y='SalePrice', kind='box')

In [ ]:
X['Electrical'].unique()
sns.catplot(data=X, x='Electrical', y='SalePrice', kind='box')

Electrical is a bit of a different case, because there is only one value and missing, and presumably all of our homes for sale have some type of electrical system. Below we walk through some steps to replace this value properly.

In [ ]:
X.query('Electrical.isna()') # Id: 1380 

In [ ]:
X['Electrical'].value_counts() # SBrkr is the most common value

In [ ]:
X.loc[1380, 'YearBuilt'] # The home with the missing value was built in 2006

# Filling the missing value with SBrkr because all homes built after the 70s seem to have it
# After some research, it looks like this type was phased in starting in the 60s
# so it's likely that this home has it as well
electrical_fill_cond = (pd.to_numeric(X['YearBuilt'], errors='coerce') > 1970) & (X['Electrical'].isna())
X.loc[electrical_fill_cond, 'Electrical'] = 'SBrkr'

Next, we will be dropping some features entirely from our dataset. Sometimes when there's a lot of missing data, it's best not to use the feature, rather than trying to guess how to fill it. Below, we write a helper function for dropping features in both the dataset, and use that to remove features with many missing values. As a reminder, those features were `MiscVal`, `PoolArea`, `3SsnPorch`, `LotArea`, and `LowQualFinSF`. I will remove `MiscVal`, `MiscFeature`, `Alley`, and `Fence` due to the high amount of missing values. The others I will keep for now because I have an idea that I will use them for feature engineering.

In [ ]:
def drop_features(features_to_drop):
    """
    Drops features (columns) from dataset.
    
    features_to_drop = an array of column names
    """
    global X
    X = X.drop(columns=features_to_drop)

In [ ]:
drop_features(['MiscFeature', 'MiscVal', 'Alley', 'Fence'])

### Remove Outliers

Using the ids of the outliers we identified during the exploration phase, we will now remove these rows from the dataset. The values that we have chosen are as below. Note that the outliers we have identified as "maybe" can be experimented with in future iterations to determine which model performs best in the evaluation phase. 

In [ ]:
outlier_ids =  [49, 186, 198, 250, 298, 314, 323, 336, 379, 496, 524, 582, 598, 636, 707, 739, 810, 935, 955, 1191, 1299, 1329]

# Remove rows with outliers 
X.drop(outlier_ids, inplace=True)

In [ ]:
# Remove rows with missing target
X.dropna(axis=0, subset=['SalePrice'], inplace=True)

### Feature Engineering

We create a helper function to easily add features to the dataset. Then, using domain knowledge we add features that may correlate with Sale Price. This is an iterative process of feature engineering, and then evaluating the effect on our models' accuracy.

In [ ]:
def add_feature(feature_name, feature_fn):
    """
    Add a new feature to the data
    
    feature_name = the name of the feature
    feature_fn = the function that returns the value, parameter is the dataframe
    
    ex:
    add_feature('HouseAge', lambda df: df['YearSold'] - df['YearBuilt'])
    """
    X[feature_name] = feature_fn(X)

In [ ]:
add_feature('HouseAge', lambda df: df['YrSold'] - df['YearBuilt'])

In [ ]:
add_feature('HouseRemodelAge', lambda df: df['YrSold'] - df['YearRemodAdd'])

In [ ]:

add_feature('TotalSF', lambda df: df['1stFlrSF'] + df['2ndFlrSF'] + df['BsmtFinSF1'] + df['BsmtFinSF2'])

In [ ]:
add_feature('TotalArea', lambda df: df['GrLivArea'] + df['TotalBsmtSF'])

In [ ]:
add_feature('TotalBaths', lambda df: df['FullBath'] + df['BsmtFullBath'] + (.5 * df['BsmtHalfBath']) + (.5 * df['HalfBath']))

In [ ]:
add_feature('TotalPorchSF', lambda df: df['OpenPorchSF'] + df['3SsnPorch'] + df['EnclosedPorch'] + df['ScreenPorch'] + df['WoodDeckSF'])

In [ ]:
# Drop the features that were used to create the new features
drop_features(['YrSold', 'YearBuilt', 'YearRemodAdd', '1stFlrSF', '2ndFlrSF', 'BsmtFinSF1', 'BsmtFinSF2', 'GrLivArea', 'TotalBsmtSF', 'BsmtFullBath', 'FullBath', 'BsmtHalfBath', 'HalfBath', 'OpenPorchSF', '3SsnPorch', 'EnclosedPorch', 'ScreenPorch', 'WoodDeckSF'])

### Feature Correlation Analysis

Next, we check for correlations between features, and drop features based on a high correlation with each other. High correlation with SalePrice is good, but high correlation with other features can skew the predictions.

In [ ]:
correlation_matrix = X.corr(numeric_only=True)
plt.figure(figsize=(20, 12))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')

In [ ]:
# GarageArea and GarageCars are highly correlated, so we'll drop GarageArea
# GarageYrBlt and YearBuilt are highly correlated, so we'll drop GarageYrBlt
# GarageCond and GarageQual are highly correlated, so we'll drop GarageCond
drop_features([ 'GarageArea', 'GarageYrBlt', 'GarageCond'])

### Other Data Preparation

Earlier we noticed that SalePrice is positively skewed. After log transformation, the distribution of SalePrice appears normal.

In [ ]:
X['SalePrice'] = np.log1p(X['SalePrice'])
sns.histplot(X, x=X['SalePrice'])

In [ ]:
# Drop features that could cause data leakage or are not useful
drop_features(['SaleType', 'SaleCondition', 'MoSold'])

### Encoding

We need to encode our categorical data into numerical formats to make these variables usable by machine learning models. Below we manually sort the categorical data columns to be either one-hot or ordinal encoded. One-hot encoding is used for nominal data where no intrinsic ordering is present, while ordinal preserves the order of categories.

In [ ]:
with pd.option_context('display.max_seq_items', None):
    print(X.dtypes[X.dtypes == 'object'].index.tolist())

In [ ]:
# Ordinal 
ode_cols = ['LotShape', 'LandContour', 'LandSlope', 'ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'HeatingQC', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageFinish', 'GarageQual', 'PavedDrive', 'PoolQC']

In [ ]:
# One hot encoding
ohe_cols = ['MSZoning', 'Street', 'Utilities', 'LotConfig', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl',
           'Exterior1st', 'Exterior2nd', 'MasVnrType', 'Foundation', 'Heating', 'CentralAir', 'Electrical', 'GarageType']

In [ ]:
# Numerical columns
num_cols = X.select_dtypes(include=['int64', 'float64']).columns
num_cols = num_cols.drop('SalePrice')

## Modeling

### Preprocessing

Before the actual modeling, we must prepare our data through a series of preprocessing steps to make sure our data is in the correct format for the machine learning algorithms to process it effectively. We start by constructing three preprocessing pipelines. For our numerical data, we implement a pipeline that first fills in any missing values with the mean (using a simple imputer) and then applies standard scaling to normalize the data. For ordinal categorical data we use a pipeline that employs an imputer with the most frequent strategy to handle missing values, followed by an ordinal encoder to convert these categories into a numerical format that preserves their inherent order. For categorical data with no intrinsic order, we create a pipeline that also starts with imputation using the most frequent strategy but then applies one-hot encoding to transform these categories into a binary matrix. With these pipelines in place, we integrate them into a single column transformer that applies each pipeline to its corresponding set of features, as categorized in the previous steps. 

In [ ]:
num_pipeline = Pipeline(steps=[
    ('impute', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler()),
])

In [ ]:
ode_pipeline = Pipeline(steps=[
    ('impute', SimpleImputer(strategy='most_frequent')),
    ('ode', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
])

In [ ]:
ohe_pipeline = Pipeline(steps=[
    ('impute', SimpleImputer(strategy='most_frequent')),
    ('ohe', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

In [ ]:
col_transformer = ColumnTransformer(transformers=[
    ('num_pipe', num_pipeline, num_cols),
    ('ode_pipe', ode_pipeline, ode_cols),
    ('ohe_pipe', ohe_pipeline, ohe_cols)
],
remainder='passthrough',
n_jobs=-1)

In [ ]:
# Separate target from predictors
y = X.SalePrice
X.drop(['SalePrice'], axis=1, inplace=True)

### Build Models

In the next phase, we construct machine learning models to predict our target variable. Our primary focus is on two models: RandomForestRegressor and XGBoost. We build a pipeline for the RandomForest model made up of two steps - preprocessing, where our previously defined column transformer is applied, and the actual RandomForest model. We use GridSearchCV to fine-tune the model by finding the best combination of hyperparameters. This process involves cross-validation to make sure the model's performance is generalizable and robust. Following a similar approach, we also apply this pipeline and grid search technique to an XGBoost model. 

In [ ]:
rfr_pipeline = Pipeline(steps=[
    ('preprocessing', col_transformer),
    ('rfr_model', RandomForestRegressor(random_state=13))
])

param_grid_rfr = {
    'rfr_model__max_depth': [5, 10, 15],
    'rfr_model__n_estimators': [100, 250, 500],
    'rfr_model__min_samples_split': [3, 5, 10]
}

rfr_cv = GridSearchCV(rfr_pipeline, param_grid_rfr, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
rfr_cv.fit(X, y)

In [ ]:
best_rfr_model = rfr_cv.best_estimator_

In [ ]:
xgb_pipeline = Pipeline(steps=[
    ('preprocessing', col_transformer),
    ('xgb_model', XGBRegressor(random_state=13))
])

param_grid_xgb = {
    'xgb_model__learning_rate': [.05, .1, .2],
    'xgb_model__n_estimators': [100, 200, 300],
    'xgb_model__max_depth': [2, 3, 4],
    'xgb_model__min_child_weight': [1, 2, 3],
    'xgb_model__gamma': [0, .1, .2],
    'xgb_model__subsample': [.8, .9, 1.0],
    'xgb_model__colsample_bytree': [.8, .9, 1.0]
}

xgb_cv = GridSearchCV(xgb_pipeline, param_grid_xgb, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
xgb_cv.fit(X, y)


In [ ]:
best_xgb_model = xgb_cv.best_estimator_

## Evaluation

In [ ]:
mse_scorer = make_scorer(mean_squared_error, greater_is_better=False)
mae_scorer = make_scorer(mean_absolute_error, greater_is_better=False)

def get_scores(pipeline, n_folds=3):
    """Return the average MSE and MAE over n_folds CV folds of XGBoost model.
    
    Keyword argument:
    pipeline == a sklearn pipeline
    n_folds == the number of CV folds
    """
    mae_scores = -1 * cross_val_score(pipeline, X, y, cv=n_folds, scoring=mae_scorer)
    mse_scores = -1 * cross_val_score(pipeline, X, y, cv=n_folds, scoring=mse_scorer)
    return {
        'mae': mae_scores.mean(),
        'mse': mse_scores.mean()
    }

In [ ]:
get_scores(best_rfr_model)

In [ ]:
get_scores(best_xgb_model)

In [ ]:
feature_importances = best_xgb_model.named_steps['xgb_model'].feature_importances_
transformed_features = best_xgb_model.named_steps['preprocessing'].get_feature_names_out()
importances = pd.Series(feature_importances, index=transformed_features)
print(importances.sort_values(ascending=False))

In [ ]:
# Number of features to visualize
top_n = 25  

# Sort importances
sorted_importances = importances.sort_values(ascending=False)

# Plot top n feature importances
plt.figure(figsize=(10, 6))
sns.barplot(x=sorted_importances[:top_n], y=sorted_importances.index[:top_n])
plt.title('Top Feature Importances')
plt.xlabel('Importance Score')
plt.ylabel('Features')
plt.show()

In [ ]:
from joblib import dump

dump(best_xgb_model, '../server/house-price.joblib')